# 벡터의 유사도(Vector Similarity)
문장이나 문서간에 유사도 측정\
문서의 수치화 방법, 차이 구하는 방법 에 따라 성능이 바뀜

## 코사인 유사도(Cosine Similarity)
DTM이나 TF-IDF행렬에서 각 행을 벡터로 생각하고 유사도 측정\
벡터의 방향을 고려하기에 문서길이를 고려x

![nn](https://wikidocs.net/images/page/24603/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.PNG)

## 유사도를 이용한 추천시스템
TF-IDF와 코사인 유사도를 이용해 영화의 줄거리에 기반해서 영화 추천\
좋아하는 영화를 입력하면 줄거리가 유사한 영화를 찾아서 추천

**사용할 데이터**
- title(제목)
- overview(줄거리)

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path

data_dir = Path('C:/Users/sinjy/jupyter_notebook/datasets/movies')
data = pd.read_csv(data_dir / 'movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
data = data.head(20000)

In [5]:
## null check 
data['overview'].isnull().sum()

135

In [6]:
## tf-idf연산에 null값은 없어야 함
## 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

In [8]:
tfidf = TfidfVectorizer(stop_words='english') # stop_words: 불용어 제거
tfidf_matrix = tfidf.fit_transform(data['overview']) # 줄거리에 대한 tf-idf
tfidf_matrix.shape # 행은 문서 개수, 열: 단어

(20000, 47487)

In [9]:
## 코사인 유사도 측정
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape # 각 문서간에 코사인 유사도

(20000, 20000)

In [10]:
title_to_index = dict(zip(data['title'], data.index)) # dict[title] = index

In [13]:
## 영화 제목을 입력하면 유사도 top 10을 리턴
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = title_to_index[title] # 영화 제목에 해당하는 index
    
    # 해당 index와 다른 index간에 유사도리스트
    sim_scores = list(enumerate(cosine_sim[idx])) 
    # 유사도에 따른 크기순 정렬, top10
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
    
    # top10의 index
    movie_indices = [idx[0] for idx in sim_scores]
    
    return data['title'].iloc[movie_indices]

In [14]:
get_recommendations('The Dark Knight Rises') # 다크나이트 라이즈

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

## 유클리드 거리(Euclidean distance)
단순 벡터간 거리를 이용한 유사도 측정

## 자카드 유사도(Jaccard similarity)
A, B: 문서를 토큰화 한 뒤에 집합\
0과 1사이의 값, 동일하면 1

$J(A, B) = \frac{|A \cap B|}{|A \cup B|}$